# RILY News Summarizer

NOTE: This is not supposed to be a thorough, flawless product ready to be sold. I'm just having some fun throwing things together quickly and learning in the process

### Step 1 - Setup

In [14]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [7]:
# Set up OpenAI API Key

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Sanity check

if api_key:
    print("OpenAI API key found!")
else:
    print("Uh oh, we don't have an API key")


OpenAI API key found!


### Step 2 - Data Gathering (templates)

In [8]:
# Fetch HTML from any provided url

def get_website_text(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.body.get_text(separator="\n", strip=True)

SEC Filings

In [9]:
def get_sec_filings():
    #  0) Get raw data
    data = get_website_text("https://ir.brileyfin.com/sec-filings")

    # 1) Split once
    entries = data.split("\n")

    # 2) Reduce elements to only include SEC Filings
    proxy_seen = False
    start_idx = None
    end_idx = None

    for i, line in enumerate(entries):
        if not proxy_seen and "Proxy & Information Statements" in line:
            proxy_seen = True
        elif proxy_seen and start_idx is None and "Format" in line:
            start_idx = i
        elif proxy_seen and "1-25" in line:
            end_idx = i  # keep updating to get the last occurrence

    # 3) Validate indices and slice; if invalid, return empty
    if not start_idx or not end_idx or start_idx > end_idx:
        return ""

    # 4) Second (partial) pass: filter out entries starting with "Open "
    filtered_entries = [line for line in entries[start_idx+1:end_idx] if not line.startswith("Open ")]

    # Combine in readable fashion
    result = []
    for i in range(0, len(filtered_entries), 3):
        form_type = filtered_entries[i]
        form_description = filtered_entries[i + 1]
        date_of_filing = filtered_entries[i + 2]
        formatted = f"Date: {date_of_filing}, Form Type: {form_type}, Description: {form_description}"
        result.append(formatted)

    return "\n".join(result)

B Riley Financial News

In [10]:
def get_financial_press_releases():
    #  0) Get raw data
    data = get_website_text("https://ir.brileyfin.com/press-releases")

    entries = data.split("\n")
    start_idx = next((i for i, line in enumerate(entries) if "Basic Search" in line), None)

    if not start_idx:
        return ""

    filtered_entries = entries[start_idx+1:start_idx+16]

    result = []
    for i in range(0, len(filtered_entries), 3):
        date = filtered_entries[i]
        title = filtered_entries[i + 1]
        description = filtered_entries[i + 2]
        formatted = f"Date: {date}, Title: {title}, Description: {description}"
        result.append(formatted)

    return "\n".join(result)

B Riley Securities News

In [11]:
def get_securities_press_releases():
    #  0) Get raw data
    data = get_website_text("https://www.brileysecurities.com/investor-relations")

    entries = data.split("\n")
    start_idx = next((i for i, line in enumerate(entries) if "Latest News" in line), None)

    if not start_idx:
        return ""

    filtered_entries = entries[start_idx+1:start_idx+13]

    result = []
    for i in range(0, len(filtered_entries), 2):
        title = filtered_entries[i]
        date = filtered_entries[i + 1]
        formatted = f"Date: {date}, Title: {title}"
        result.append(formatted)

    return "\n".join(result)

### Step 3 - Fetch the Data

In [16]:
# Fetch all data we need
filings_input = get_sec_filings()
financial_input = get_financial_press_releases()
securities_input = get_securities_press_releases()

if filings_input and financial_input and securities_input:
    print("Success!")
else:
    if not filings_input:
        print("Unable to gather SEC filings")
    if not financial_input:
        print("Unable to gather financial news")
    if not securities_input:
        print("Unable to gather securities news")

Success!


### Step 4 - Summarize and Analyze

In [ ]:
from datetime import date

NUM_DAYS_FOR_SUMMARY = 14

today = date.today()

# Step 1: Create your prompts

system_prompt = f"""You are an expert financial researcher. You are researching B Riley Financial, described as \
\"a diversified financial services company that provides investment banking, wealth management, and financial consulting services to corporations, financial sponsors, and high-net-worth individuals\". \
It is the parent company to B Riley Securities, which provides a range of investment banking services, including M&A advisory, IPOs, institutional private placements, and corporate restructuring.

You will, be given 3 sources of information:
1. The company's SEC filings
2. B Riley Financial's most recent announcements
3. B Riley securities' most recent announcements

Generate a summary of the news from the past {NUM_DAYS_FOR_SUMMARY} days. What announcements have been made? Has the company made any new filings? What might these announcements mean for the well-being of the company?

Output Format:
\"
### Summary of Recent B Riley Financial News (Past <x> Days)

**Recent Filings:**
1. **<form_type> (<date_filed>)**:
- <description of event>
- <analysis>
...

**Recent Announcements from B Riley Financial:**
1. **<event_name> (<date>)**:
- <description of event>
- <analysis>
...

**Recent Announcements from B Riley Securities:**
1. **<event_name> (<date>)**:
- <description of event>
- <analysis>
...
\"

Guidelines:
- Use spartan tone
- Be concise and objective
- Give an analysis of each piece of news individually
- ONLY INCLUDE EVENTS THAT OCCURRED WITHIN THE PAST {NUM_DAYS_FOR_SUMMARY} DAYS (today is {f"{today.strftime('%B')} {today.day}, {today.year}"})
"""

user_prompt = f"""
Most recent company filings:
{filings_input}

Recent B Riley Financial News:
{financial_input}

Recent B Riley Securities News:
{securities_input}
"""

# Step 2: Make the messages list

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

# Step 3: Call OpenAI

openai = OpenAI()
response = openai.chat.completions.create(
    model = "gpt-4o-mini",
    messages = messages
)

# Step 4: print the result

display(Markdown(response.choices[0].message.content))

### Summary of Recent B Riley Financial News (Past 14 Days)

**Recent Filings:**
1. **Form 10-K (Sep 19, 2025)**:
   - The company filed its annual report, providing a comprehensive overview of its financial performance over the past year.
   - **Analysis**: This filing is crucial for investors and stakeholders as it outlines B Riley's financial health, performance metrics, and strategic outlook. It may impact investor confidence and future investment decisions.
   
2. **Form 8-K (Sep 12, 2025)**:
   - Reported an unscheduled material event or corporate event.
   - **Analysis**: Details of the event were not specified in the provided data. However, such filings typically indicate significant developments that could affect the company's operations or financial condition, warranting closer scrutiny by analysts and investors.

**Recent Announcements from B Riley Financial:**
1. **BDO USA for 2025 Audit (Sep 9, 2025)**:
   - The company announced the appointment of BDO USA, P.C. as its independent auditor for the 2025 fiscal year.
   - **Analysis**: Engaging a reputable firm for auditing may enhance the credibility of financial disclosures and improve investor trust. This is particularly important given the recent financial challenges the company has faced.

2. **Notification of Delinquency with Nasdaq (Aug 22, 2025)**:
   - The company received a delinquency notification letter from Nasdaq, indicating compliance issues regarding listing standards.
   - **Analysis**: This notification could pose risks to the company's listing status, which could adversely affect its market reputation and stock performance. It is critical for B Riley to quickly resolve these compliance issues.

3. **Preliminary Q4 and Full Year 2024 Results (Aug 13, 2025)**:
   - The company shared early results, projecting net income between $125 million and $145 million for the first half of 2025.
   - **Analysis**: Providing such guidance suggests confidence in financial recovery. However, actual performance will need to align with these estimates to maintain investor support.

**Recent Announcements from B Riley Securities:**
1. **Lead Bookrunner for OTG Acquisition Corp. IPO ($230 Million, Sep 17, 2025)**:
   - B Riley Securities served as lead bookrunner for a significant IPO.
   - **Analysis**: This role demonstrates B Riley Securities' strong position in the investment banking sector, potentially leading to increased revenues and enhancing the firm's market presence.

2. **Financial Advisor to Smile Design Dentistry (Sep 9, 2025)**:
   - Acted as lead financial advisor in a strategic transaction with Heartland Dental.
   - **Analysis**: This advisory role may bolster revenue streams and showcase the firm’s capabilities in closing important transactions, thereby contributing to its overall reputation.

3. **BDC & CLO Investor Conference (Aug 27, 2025)**:
   - B Riley Securities announced a conference scheduled for October, focusing on business development companies and collateralized loan obligations.
   - **Analysis**: Hosting an investor conference reflects proactive engagement with the investment community, which could enhance relationships with current and future clients.

Overall, B Riley Financial's recent activities indicate a mix of strategic appointments and necessary compliance issues. While there are proactive efforts to bolster investor confidence, management must address Nasdaq compliance concerns to safeguard its financial standing and reputation in capital markets.